In [1]:
import pandas as pd
import re
import os
import datetime
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
import multiprocessing
import sys
import requests
import csv
import time
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopandas 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Выгрузка FAA

Выгрузим датасэты по годам и объединим их в один датасэт

In [6]:
# Создаем список годов
years = [2018, 2019, 2020, 2021, 2022]

# Путь к папке с файлами
data_folder = "C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/"

# Создаем пустой список для датасетов
data_list = []

# Загружаем данные из файлов и добавьте их в список
for year in years:
    file_path = os.path.join(data_folder, f"{year}.xlsx")
    data = pd.read_excel(file_path)
    data_list.append(data)

# Объединим датасеты в один
data_faa = pd.concat(data_list, ignore_index=True)

# Выведем информацию о полученном датасете
data_faa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78028 entries, 0 to 78027
Columns: 102 entries, INDX_NR to TRANSFER
dtypes: bool(39), float64(19), int64(3), object(41)
memory usage: 40.4+ MB


In [7]:
data_faa.to_csv("C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/data_faa_full.csv", sep=',', index=False, quoting=csv.QUOTE_ALL)

In [8]:
data_faa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78028 entries, 0 to 78027
Columns: 102 entries, INDX_NR to TRANSFER
dtypes: bool(39), float64(19), int64(3), object(41)
memory usage: 40.4+ MB


In [9]:
data_faa.describe()

,INDX_NR,INCIDENT_MONTH,INCIDENT_YEAR,AMO,EMA,EMO,AC_MASS,NUM_ENGS,ENG_1_POS,ENG_2_POS,...,HEIGHT,SPEED,DISTANCE,AOS,COST_REPAIRS,COST_OTHER,COST_REPAIRS_INFL_ADJ,COST_OTHER_INFL_ADJ,NR_INJURIES,NR_FATALITIES
count,7.802800e+04,78028.000000,78028.000000,43353.000000,45426.000000,42834.000000,51072.000000,50992.000000,50755.000000,46487.000000,...,32469.000000,18198.00000,55372.000000,3469.000000,7.390000e+02,1.126000e+03,7.390000e+02,1.126000e+03,28.000000,6.000000
mean,1.081516e+06,7.289217,2020.004396,24.419786,17.055717,6.043447,3.544545,1.951345,2.554014,2.242735,...,898.331085,143.48324,1.017484,50.305930,1.295101e+05,2.517740e+04,1.410831e+05,2.662100e+04,1.142857,1.666667
std,1.894820e+05,2.695747,1.461602,21.546882,9.610621,9.369055,0.912351,0.351618,2.146975,1.859769,...,1841.426075,49.01840,3.849539,319.032392,6.898597e+05,3.799720e+05,7.457443e+05,3.816198e+05,0.356348,0.816497
min,8.013900e+05,1.000000,2018.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.00000,0.000000,0.050000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000
25%,9.335538e+05,5.000000,2019.000000,7.000000,10.000000,1.000000,3.000000,2.000000,1.000000,1.000000,...,0.000000,120.00000,0.000000,1.000000,2.500000e+03,2.000000e+02,2.860000e+03,2.300000e+02,1.000000,1.000000
50%,1.090288e+06,8.000000,2020.000000,16.000000,13.000000,4.000000,4.000000,2.000000,1.000000,1.000000,...,50.000000,140.00000,0.000000,2.000000,1.200000e+04,5.000000e+02,1.357200e+04,5.660000e+02,1.000000,1.500000
75%,1.253920e+06,9.000000,2021.000000,42.000000,22.000000,6.000000,4.000000,2.000000,5.000000,5.000000,...,1000.000000,160.00000,0.000000,12.000000,5.000000e+04,2.500000e+03,5.655000e+04,2.860000e+03,1.000000,2.000000
max,1.449554e+06,12.000000,2022.000000,98.000000,46.000000,90.000000,5.000000,4.000000,7.000000,6.000000,...,29000.000000,1250.00000,99.000000,8938.000000,1.104179e+07,1.250000e+07,1.192513e+07,1.250000e+07,2.000000,3.000000


Переименуй столбец, так как у него есть лишний столбец, так будет проще дальше работать с этим столбцом

In [10]:
data_faa = data_faa.rename(columns = {'INCIDENT_DATE ': 'INCIDENT_DATE'})

Проверим датасэт на дубликаты

In [11]:
data_faa.duplicated().sum()

0

In [12]:
database = data_faa

Поработаем со столбцом 'TIME'

In [13]:
database.head(10)

,INDX_NR,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME,TIME_OF_DAY,AIRPORT_ID,AIRPORT,LATITUDE,LONGITUDE,...,NR_INJURIES,NR_FATALITIES,COMMENTS,REPORTER_NAME,REPORTER_TITLE,SOURCE,PERSON,LUPDATE,IMAGE,TRANSFER
0,817702,2018-12-31,12,2018,,NaN,KLAL,LAKELAND LINDER REGIONAL,27.98892,-82.01856,...,NaN,NaN,"XXXX-XX-XX-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Carcass Found,2019-01-24,False,False
1,817703,2018-12-31,12,2018,,NaN,KMFE,MC ALLEN MILLER INTL,26.17583,-98.23861,...,NaN,NaN,XXXX-XX-XX-XXXXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,FAA Form 5200-7-E,Carcass Found,2019-01-24,False,False
2,817704,2018-12-31,12,2018,,NaN,KLAX,LOS ANGELES INTL,33.94254,-118.40807,...,NaN,NaN,"XXXX-XX-XX-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Carcass Found,2019-01-24,False,False
3,817705,2018-12-31,12,2018,,NaN,KAUS,AUSTIN-BERGSTROM INTL,30.19453,-97.66987,...,NaN,NaN,"XXXX-XX-XX-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Carcass Found,2019-01-24,False,False
4,817707,2018-12-31,12,2018,,NaN,PHJH,KAPALUA ARPT,20.96294,-156.67303,...,NaN,NaN,XXXX-XX-XX-XXXXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,FAA Form 5200-7-E,Carcass Found,2019-01-24,False,False
5,817711,2018-12-31,12,2018,,NaN,KSLC,SALT LAKE CITY INTL,40.78839,-111.97777,...,NaN,NaN,XXXX-XX-XX-XXXXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,FAA Form 5200-7-E,Carcass Found,2019-01-24,False,False
6,817909,2018-12-31,12,2018,,NaN,KSMF,SACRAMENTO INTL,38.69542,-121.59077,...,NaN,NaN,FAA DAILY ALERT BULLETIN /Legacy Record=XXXXXX/,REDACTED,REDACTED,Daily Report,Tower,2019-02-20,False,False
7,818272,2018-12-31,12,2018,10:10,Day,KSEA,SEATTLE-TACOMA INTL,47.44898,-122.30931,...,NaN,NaN,XXXX-X-X-XXXXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,FAA Form 5200-7-E,Pilot,2019-03-18,False,False
8,818275,2018-12-31,12,2018,12:07,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,NaN,NaN,"XXXX-X-X-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Pilot,2019-03-18,False,False
9,818288,2018-12-31,12,2018,20:54,Night,KLAX,LOS ANGELES INTL,33.94254,-118.40807,...,NaN,NaN,"XXXX-X-X-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Airport Operations,2019-03-18,False,False


### INCIDENT_DATE

In [14]:
# Попытаемся преобразовать столбец "Your_Column_Name" в тип данных datetime с ошибками 'coerce'
database['TIME'] = pd.to_datetime(database['TIME'], errors='coerce')

# Найдем строки, где значение столбца не удалось преобразовать
invalid_values = database[database['TIME'].isna()]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14544\683374573.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  database['TIME'] = pd.to_datetime(database['TIME'], errors='coerce')


In [15]:
# Посчитайте долю не преобразованных значений в столбце "Combined_Column"
percentage_not_converted = (invalid_values['TIME'].isna().sum() / len(database)) * 100

# Выведите долю не преобразованных значений
print(f'Доля не преобразованных значений: {percentage_not_converted:.2f}%')

Доля не преобразованных значений: 19.09%


Так как нам нужно знать точное время, чтобы определить метеостанцию, то удалим все значение, у которых отсутствует показатель времени.

In [16]:
# Получим индексы строк с проблемными значениями
indexes_to_drop = invalid_values.index

# Удалим строки с проблемными значениями из общей таблицы
database = database.drop(indexes_to_drop)

In [17]:
database.info(10)

<class 'pandas.core.frame.DataFrame'>
Index: 63130 entries, 7 to 78027
Data columns (total 102 columns):
 #    Column                 Dtype         
---   ------                 -----         
 0    INDX_NR                int64         
 1    INCIDENT_DATE          object        
 2    INCIDENT_MONTH         int64         
 3    INCIDENT_YEAR          int64         
 4    TIME                   datetime64[ns]
 5    TIME_OF_DAY            object        
 6    AIRPORT_ID             object        
 7    AIRPORT                object        
 8    LATITUDE               object        
 9    LONGITUDE              object        
 10   RUNWAY                 object        
 11   STATE                  object        
 12   FAAREGION              object        
 13   LOCATION               object        
 14   ENROUTE_STATE          object        
 15   OPID                   object        
 16   OPERATOR               object        
 17   REG                    object        
 18   FLT      

In [18]:
database.head(10)

,INDX_NR,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME,TIME_OF_DAY,AIRPORT_ID,AIRPORT,LATITUDE,LONGITUDE,...,NR_INJURIES,NR_FATALITIES,COMMENTS,REPORTER_NAME,REPORTER_TITLE,SOURCE,PERSON,LUPDATE,IMAGE,TRANSFER
7,818272,2018-12-31,12,2018,2023-09-20 10:10:00,Day,KSEA,SEATTLE-TACOMA INTL,47.44898,-122.30931,...,NaN,NaN,XXXX-X-X-XXXXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,FAA Form 5200-7-E,Pilot,2019-03-18,False,False
8,818275,2018-12-31,12,2018,2023-09-20 12:07:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,NaN,NaN,"XXXX-X-X-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Pilot,2019-03-18,False,False
9,818288,2018-12-31,12,2018,2023-09-20 20:54:00,Night,KLAX,LOS ANGELES INTL,33.94254,-118.40807,...,NaN,NaN,"XXXX-X-X-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Airport Operations,2019-03-18,False,False
10,818618,2018-12-31,12,2018,2023-09-20 15:53:00,Day,KDAL,DALLAS LOVE FIELD ARPT,32.84711,-96.85177,...,NaN,NaN,MOR: DAL-M-XXXX/XX/XX-XXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,MOR,Tower,2019-07-27,False,False
11,819264,2018-12-31,12,2018,2023-09-20 05:28:00,Night,KORD,CHICAGO O'HARE INTL ARPT,41.9796,-87.90446,...,NaN,NaN,MOR: ORD-M-XXXX/XX/XX-XXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,MOR,Tower,2019-07-27,False,False
12,819300,2018-12-31,12,2018,2023-09-20 00:04:00,Night,KPHX,PHOENIX SKY HARBOR INTL ARPT,33.43417,-112.00806,...,NaN,NaN,MOR: PHX-M-XXXX/XX/XX-XXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,MOR,Tower,2019-07-27,False,False
13,822577,2018-12-31,12,2018,2023-09-20 16:30:00,Day,ZZZZ,UNKNOWN,NaN,NaN,...,NaN,NaN,DAILY ALERT BULLETIN; MOR: GTR-M-XXXX/XX/XX-XX...,REDACTED,REDACTED,Multiple,Tower,2019-01-24,False,False
16,988134,2018-12-31,12,2018,2023-09-20 08:07:00,Day,ZZZZ,UNKNOWN,NaN,NaN,...,NaN,NaN,added State and location,REDACTED,REDACTED,FAA Form 5200-7-E,Air Transport Operations,2020-06-03,False,False
17,1278542,2018-12-31,12,2018,2023-09-20 13:48:00,Day,KMCI,KANSAS CITY INTL,39.29761,-94.71391,...,NaN,NaN,*** Strike Report: XXXX-XX-XX-XXXXXX-RX( Repor...,REDACTED,REDACTED,Multiple,Airport Operations,2022-08-03,False,False
18,1278543,2018-12-31,12,2018,2023-09-20 12:41:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,NaN,NaN,*** Strike Report: XXXX-X-X-XXXXXX-RX( Report ...,REDACTED,REDACTED,Multiple,Tower,2022-08-03,False,False


In [19]:
# Создадим новый столбец, объединяя значения из столбцов "Год", "Месяц" и "Время"

database['INCIDENT_DATE'] = pd.to_datetime(database['INCIDENT_YEAR'].astype(str) + '-' +
                                        database['INCIDENT_MONTH'].astype(str) + '-' +
                                        database['TIME'].dt.strftime('%d %H:%M:%S'))

In [20]:
database.head(10)

,INDX_NR,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME,TIME_OF_DAY,AIRPORT_ID,AIRPORT,LATITUDE,LONGITUDE,...,NR_INJURIES,NR_FATALITIES,COMMENTS,REPORTER_NAME,REPORTER_TITLE,SOURCE,PERSON,LUPDATE,IMAGE,TRANSFER
7,818272,2018-12-20 10:10:00,12,2018,2023-09-20 10:10:00,Day,KSEA,SEATTLE-TACOMA INTL,47.44898,-122.30931,...,NaN,NaN,XXXX-X-X-XXXXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,FAA Form 5200-7-E,Pilot,2019-03-18,False,False
8,818275,2018-12-20 12:07:00,12,2018,2023-09-20 12:07:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,NaN,NaN,"XXXX-X-X-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Pilot,2019-03-18,False,False
9,818288,2018-12-20 20:54:00,12,2018,2023-09-20 20:54:00,Night,KLAX,LOS ANGELES INTL,33.94254,-118.40807,...,NaN,NaN,"XXXX-X-X-XXXXXX, RX /Legacy Record=XXXXXX/",REDACTED,REDACTED,FAA Form 5200-7-E,Airport Operations,2019-03-18,False,False
10,818618,2018-12-20 15:53:00,12,2018,2023-09-20 15:53:00,Day,KDAL,DALLAS LOVE FIELD ARPT,32.84711,-96.85177,...,NaN,NaN,MOR: DAL-M-XXXX/XX/XX-XXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,MOR,Tower,2019-07-27,False,False
11,819264,2018-12-20 05:28:00,12,2018,2023-09-20 05:28:00,Night,KORD,CHICAGO O'HARE INTL ARPT,41.9796,-87.90446,...,NaN,NaN,MOR: ORD-M-XXXX/XX/XX-XXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,MOR,Tower,2019-07-27,False,False
12,819300,2018-12-20 00:04:00,12,2018,2023-09-20 00:04:00,Night,KPHX,PHOENIX SKY HARBOR INTL ARPT,33.43417,-112.00806,...,NaN,NaN,MOR: PHX-M-XXXX/XX/XX-XXXX /Legacy Record=XXXXXX/,REDACTED,REDACTED,MOR,Tower,2019-07-27,False,False
13,822577,2018-12-20 16:30:00,12,2018,2023-09-20 16:30:00,Day,ZZZZ,UNKNOWN,NaN,NaN,...,NaN,NaN,DAILY ALERT BULLETIN; MOR: GTR-M-XXXX/XX/XX-XX...,REDACTED,REDACTED,Multiple,Tower,2019-01-24,False,False
16,988134,2018-12-20 08:07:00,12,2018,2023-09-20 08:07:00,Day,ZZZZ,UNKNOWN,NaN,NaN,...,NaN,NaN,added State and location,REDACTED,REDACTED,FAA Form 5200-7-E,Air Transport Operations,2020-06-03,False,False
17,1278542,2018-12-20 13:48:00,12,2018,2023-09-20 13:48:00,Day,KMCI,KANSAS CITY INTL,39.29761,-94.71391,...,NaN,NaN,*** Strike Report: XXXX-XX-XX-XXXXXX-RX( Repor...,REDACTED,REDACTED,Multiple,Airport Operations,2022-08-03,False,False
18,1278543,2018-12-20 12:41:00,12,2018,2023-09-20 12:41:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,NaN,NaN,*** Strike Report: XXXX-X-X-XXXXXX-RX( Report ...,REDACTED,REDACTED,Multiple,Tower,2022-08-03,False,False


### LATITUDE и LONGITUDE

In [21]:
database.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63130 entries, 7 to 78027
Columns: 102 entries, INDX_NR to TRANSFER
dtypes: bool(39), datetime64[ns](2), float64(19), int64(3), object(39)
memory usage: 33.2+ MB


Преобразуем столбцы 'LATITUDE' и 'LONGITUDE' в формат float

In [22]:
database = database[pd.to_numeric(database['LATITUDE'], errors='coerce').notnull() & 
                            pd.to_numeric(database['LONGITUDE'], errors='coerce').notnull()]

In [23]:
database.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55527 entries, 7 to 78027
Columns: 102 entries, INDX_NR to TRANSFER
dtypes: bool(39), datetime64[ns](2), float64(19), int64(3), object(39)
memory usage: 29.2+ MB


Теперь заново преобразуем эти столбцы в формат float

In [24]:
database['LATITUDE'] = database['LATITUDE'].astype(float)
database['LONGITUDE'] = database['LONGITUDE'].astype(float)

In [25]:
database.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55527 entries, 7 to 78027
Columns: 102 entries, INDX_NR to TRANSFER
dtypes: bool(39), datetime64[ns](2), float64(21), int64(3), object(37)
memory usage: 29.2+ MB


In [26]:
database.to_csv("C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/database.csv")

## Выгрузка NCEI

In [27]:
data_ncei = pd.read_csv("C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/isd-history.csv", 
                        converters={'USAF':str, 'WBAN':str})

In [28]:
data_ncei.head(30)

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,NaN,NaN,NaN,0.000,0.000,7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,NaN,NaN,0.000,0.000,7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,NaN,NaN,0.000,0.000,7070.0,20140923,20150926
3,008260,99999,WXPOD8270,NaN,NaN,NaN,0.000,0.000,0.0,20050101,20120731
4,008268,99999,WXPOD8278,AF,NaN,NaN,32.950,65.567,1156.7,20100519,20120323
5,008307,99999,WXPOD 8318,AF,NaN,NaN,0.000,0.000,8318.0,20100421,20100421
6,008411,99999,XM20,NaN,NaN,NaN,NaN,NaN,NaN,20160217,20160217
7,008414,99999,XM18,NaN,NaN,NaN,NaN,NaN,NaN,20160216,20160217
8,008415,99999,XM21,NaN,NaN,NaN,NaN,NaN,NaN,20160217,20200314
9,008418,99999,XM24,NaN,NaN,NaN,NaN,NaN,NaN,20160217,20160217


In [29]:
data_ncei.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29637 entries, 0 to 29636
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USAF          29637 non-null  object 
 1   WBAN          29637 non-null  object 
 2   STATION NAME  28847 non-null  object 
 3   CTRY          28742 non-null  object 
 4   STATE         6625 non-null   object 
 5   ICAO          10819 non-null  object 
 6   LAT           28451 non-null  float64
 7   LON           28450 non-null  float64
 8   ELEV(M)       28347 non-null  float64
 9   BEGIN         29637 non-null  int64  
 10  END           29637 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 2.5+ MB


In [30]:
# Объединяем столбцы USAF и WBAN без пробела в новый столбец 'USAF_WBAN'
data_ncei['USAF_WBAN'] = data_ncei['USAF'].astype(str) + data_ncei['WBAN'].astype(str)

In [31]:
# Выбираем только нужные столбцы, включая объединенный 'WBAN_USAF'
selected_columns = ['USAF_WBAN', 'LAT', 'LON', 'BEGIN', 'END']
data_ncei = data_ncei[selected_columns]

In [32]:
data_ncei.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29637 entries, 0 to 29636
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   USAF_WBAN  29637 non-null  object 
 1   LAT        28451 non-null  float64
 2   LON        28450 non-null  float64
 3   BEGIN      29637 non-null  int64  
 4   END        29637 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 1.1+ MB


In [33]:
data_ncei = data_ncei.dropna().drop_duplicates()

In [34]:
data_ncei.head(10)

,USAF_WBAN,LAT,LON,BEGIN,END
0,00701899999,0.000,0.000,20110309,20130730
1,00702699999,0.000,0.000,20120713,20170822
2,00707099999,0.000,0.000,20140923,20150926
3,00826099999,0.000,0.000,20050101,20120731
4,00826899999,32.950,65.567,20100519,20120323
5,00830799999,0.000,0.000,20100421,20100421
11,01001099999,70.933,-8.667,19310101,20230902
13,01001499999,59.792,5.341,19861120,20230901
14,01001599999,61.383,5.867,19870117,19971231
15,01001699999,64.850,11.233,19870116,19910806


Также в исходной таблице видели значения LAT и LON 0.000

Проверим эти значения

In [35]:
data_ncei_check = data_ncei[(data_ncei['LAT'] == 0.0) & (data_ncei['LON'] == 0.0)]

In [36]:
data_ncei_check

,USAF_WBAN,LAT,LON,BEGIN,END
0,00701899999,0.0,0.0,20110309,20130730
1,00702699999,0.0,0.0,20120713,20170822
2,00707099999,0.0,0.0,20140923,20150926
3,00826099999,0.0,0.0,20050101,20120731
5,00830799999,0.0,0.0,20100421,20100421
...,...,...,...,...,...
24934,89979099999,0.0,0.0,19760601,19760713
27139,99118099999,0.0,0.0,20011104,20011104
27140,99119099999,0.0,0.0,20020329,20020329
27141,99120099999,0.0,0.0,20020516,20020516


Так как много метеостанций находятся в одной точке с координатами 0.0 0.0 скорее всего это ошибка при обратке данных, которые попадают в базу. Удалим эти данные

In [37]:
data_ncei = data_ncei[(data_ncei['LAT'] != 0.0) & (data_ncei['LON'] != 0.0)]

In [38]:
data_ncei.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28056 entries, 4 to 29636
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   USAF_WBAN  28056 non-null  object 
 1   LAT        28056 non-null  float64
 2   LON        28056 non-null  float64
 3   BEGIN      28056 non-null  int64  
 4   END        28056 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 1.3+ MB


In [39]:
data_ncei.head(10)

,USAF_WBAN,LAT,LON,BEGIN,END
4,00826899999,32.950,65.567,20100519,20120323
11,01001099999,70.933,-8.667,19310101,20230902
13,01001499999,59.792,5.341,19861120,20230901
14,01001599999,61.383,5.867,19870117,19971231
15,01001699999,64.850,11.233,19870116,19910806
16,01001799999,59.980,2.250,19880320,19971226
17,01002099999,80.050,16.250,19861109,20230901
18,01003099999,77.000,15.500,19850601,20230902
19,01004099999,78.917,11.933,19730101,19970801
20,01005099999,78.067,13.633,19310103,20041030


### 'BEGIN' и 'END'

Обработаем столбцы 'BEGIN' и 'END', чтобы в дальнейшем определить работала ли станция в моменты инцидента или нет

In [40]:
# Преобразуем столбцы 'BEGIN' и 'END' в формат даты

data_ncei['BEGIN'] = pd.to_datetime(data_ncei['BEGIN'], format='%Y%m%d')
data_ncei['END'] = pd.to_datetime(data_ncei['END'], format='%Y%m%d')

In [41]:
data_ncei.head(10)

,USAF_WBAN,LAT,LON,BEGIN,END
4,00826899999,32.950,65.567,2010-05-19,2012-03-23
11,01001099999,70.933,-8.667,1931-01-01,2023-09-02
13,01001499999,59.792,5.341,1986-11-20,2023-09-01
14,01001599999,61.383,5.867,1987-01-17,1997-12-31
15,01001699999,64.850,11.233,1987-01-16,1991-08-06
16,01001799999,59.980,2.250,1988-03-20,1997-12-26
17,01002099999,80.050,16.250,1986-11-09,2023-09-01
18,01003099999,77.000,15.500,1985-06-01,2023-09-02
19,01004099999,78.917,11.933,1973-01-01,1997-08-01
20,01005099999,78.067,13.633,1931-01-03,2004-10-30


Так как мы выгружаем данные с 2018 года, сразу отфильтруем ты метеостанции которые перестали работать к 2018 году

In [42]:
data_ncei = data_ncei[data_ncei['END'] >= '2018-01-01']

In [43]:
data_ncei.head(10)

,USAF_WBAN,LAT,LON,BEGIN,END
11,01001099999,70.933,-8.667,1931-01-01,2023-09-02
13,01001499999,59.792,5.341,1986-11-20,2023-09-01
17,01002099999,80.050,16.250,1986-11-09,2023-09-01
18,01003099999,77.000,15.500,1985-06-01,2023-09-02
21,01006099999,78.250,22.817,1973-01-01,2023-09-01
22,01007099999,78.917,11.933,1973-01-06,2023-09-02
24,01008099999,78.246,15.466,1975-09-29,2023-09-02
25,01009099999,80.650,25.000,1955-01-01,2023-09-02
26,01010099999,69.293,16.144,1931-01-03,2023-09-02
27,01011099999,80.067,31.500,1986-11-18,2023-09-01


In [44]:
data_ncei.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14446 entries, 11 to 29636
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   USAF_WBAN  14446 non-null  object        
 1   LAT        14446 non-null  float64       
 2   LON        14446 non-null  float64       
 3   BEGIN      14446 non-null  datetime64[ns]
 4   END        14446 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 677.2+ KB


В итоге получили датасэт который в дальнейшим мы объединим с датасэтом FAA

## Поиск ближайшей метеостанции

Разделим поиск ближайшей метеостанции по годам.

### Данные за 2018

In [45]:
# Создаем DataFrame для данных 2018-2022 годов
df_results = database[(database['INCIDENT_DATE'].dt.year >= 2018) & (database['INCIDENT_DATE'].dt.year <= 2022)].copy()

# Проходим по годам от 2018 до 2022 включительно
for year in range(2018, 2023):
    # Создаем пустой список для результатов текущего года
    results_year = []

    # Фильтруем инциденты только для текущего года
    df_year = df_results[df_results['INCIDENT_DATE'].dt.year == year].copy()

    # Проходимся по инцидентам в текущем году
    for _, incident in tqdm(df_year.iterrows(), total=len(df_year)):
        incident_coords = Point(incident['LONGITUDE'], incident['LATITUDE'])
        min_distance = float('inf')
        nearest_station = None

        # Фильтруем метеостанции, которые работали в момент инцидента
        relevant_stations = data_ncei[
            (data_ncei['BEGIN'] <= incident['INCIDENT_DATE']) &
            (data_ncei['END'] >= incident['INCIDENT_DATE'])
        ]

        for _, station in relevant_stations.iterrows():
            station_coords = Point(station['LON'], station['LAT'])
            distance = incident_coords.distance(station_coords)

            if distance < min_distance:
                min_distance = distance
                nearest_station = station

        # Проверяем, удалось ли найти ближайшую метеостанцию и подходит ли она по времени
        if nearest_station is not None and nearest_station['BEGIN'] <= incident['INCIDENT_DATE'] <= nearest_station['END']:
            result = incident.to_dict()  # Преобразуем инцидент в словарь
            # Добавляем информацию о ближайшей метеостанции к словарю результатов
            result['NEAREST_STATION_LAT'] = nearest_station['LAT']
            result['NEAREST_STATION_LON'] = nearest_station['LON']
            result['NEAREST_STATION_BEGIN'] = nearest_station['BEGIN']
            result['NEAREST_STATION_END'] = nearest_station['END']
            result['USAF_WBAN'] = nearest_station['USAF_WBAN']
            results_year.append(result)
        else:
            # Если не удалось найти подходящую станцию, добавляем пустые значения
            result = incident.to_dict()
            result['NEAREST_STATION_LAT'] = None
            result['NEAREST_STATION_LON'] = None
            result['NEAREST_STATION_BEGIN'] = None
            result['NEAREST_STATION_END'] = None
            result['USAF_WBAN'] = None
            results_year.append(result)

    # Создаем датафрейм из результатов текущего года
    df_results_year = pd.DataFrame(results_year)

    # Сохраняем результаты текущего года в CSV
    df_results_year.to_csv(f"C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/FAA+NCEI/result_{year}_full.csv", sep=',', index=False, quoting=csv.QUOTE_ALL)

100%|██████████████████████████████████████████████████████████████████████████| 13805/13805 [3:00:36<00:00,  1.27it/s]


In [63]:
# Создаем список годов
years = [2018, 2019, 2020, 2021, 2022]

# Путь к папке с файлами
data_folder = "C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/FAA+NCEI"

# Создаем пустой список для датасетов
data_list = []

# Загружаем данные из файлов и добавьте их в список
for year in years:
    file_path = os.path.join(data_folder, f"result_{year}_full.csv")
    data = pd.read_csv(file_path)
    data_list.append(data)

# Объединим датасеты в один
data_faa_ncei = pd.concat(data_list, ignore_index=True)

# Выведем информацию о полученном датасете
data_faa_ncei.info()

data_faa_ncei.to_csv("C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/FAA+NCEI/data_faa_ncei.csv", index=False, sep=',', quoting=csv.QUOTE_ALL)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14544\83167653.py:13: DtypeWarning: Columns (13,14,106) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14544\83167653.py:13: DtypeWarning: Columns (13,87) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14544\83167653.py:13: DtypeWarning: Columns (14,106) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14544\83167653.py:13: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55527 entries, 0 to 55526
Columns: 107 entries, INDX_NR to USAF_WBAN
dtypes: bool(39), float64(23), int64(3), object(42)
memory usage: 30.9+ MB


In [64]:
data_faa_ncei.head(10)

,INDX_NR,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME,TIME_OF_DAY,AIRPORT_ID,AIRPORT,LATITUDE,LONGITUDE,...,SOURCE,PERSON,LUPDATE,IMAGE,TRANSFER,NEAREST_STATION_LAT,NEAREST_STATION_LON,NEAREST_STATION_BEGIN,NEAREST_STATION_END,USAF_WBAN
0,818272,2018-12-20 10:10:00,12,2018,2023-09-20 10:10:00,Day,KSEA,SEATTLE-TACOMA INTL,47.44898,-122.30931,...,FAA Form 5200-7-E,Pilot,2019-03-18,False,False,47.445,-122.314,1948-01-01,2023-09-03,72793024233
1,818275,2018-12-20 12:07:00,12,2018,2023-09-20 12:07:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,FAA Form 5200-7-E,Pilot,2019-03-18,False,False,25.788,-80.317,1973-01-01,2023-09-03,72202012839
2,818288,2018-12-20 20:54:00,12,2018,2023-09-20 20:54:00,Night,KLAX,LOS ANGELES INTL,33.94254,-118.40807,...,FAA Form 5200-7-E,Airport Operations,2019-03-18,False,False,33.938,-118.387,1944-01-01,2023-09-03,72295023174
3,818618,2018-12-20 15:53:00,12,2018,2023-09-20 15:53:00,Day,KDAL,DALLAS LOVE FIELD ARPT,32.84711,-96.85177,...,MOR,Tower,2019-07-27,False,False,32.838,-96.836,1946-07-26,2023-09-03,72258013960
4,819264,2018-12-20 05:28:00,12,2018,2023-09-20 05:28:00,Night,KORD,CHICAGO O'HARE INTL ARPT,41.97960,-87.90446,...,MOR,Tower,2019-07-27,False,False,41.960,-87.932,1946-10-01,2023-09-03,72530094846
5,819300,2018-12-20 00:04:00,12,2018,2023-09-20 00:04:00,Night,KPHX,PHOENIX SKY HARBOR INTL ARPT,33.43417,-112.00806,...,MOR,Tower,2019-07-27,False,False,33.428,-112.004,1973-01-01,2023-09-03,72278023183
6,1278542,2018-12-20 13:48:00,12,2018,2023-09-20 13:48:00,Day,KMCI,KANSAS CITY INTL,39.29761,-94.71391,...,Multiple,Airport Operations,2022-08-03,False,False,39.297,-94.731,1973-01-01,2023-09-03,72446003947
7,1278543,2018-12-20 12:41:00,12,2018,2023-09-20 12:41:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,Multiple,Tower,2022-08-03,False,False,25.788,-80.317,1973-01-01,2023-09-03,72202012839
8,1278544,2018-12-20 08:15:00,12,2018,2023-09-20 08:15:00,Day,KSMF,SACRAMENTO INTL,38.69542,-121.59077,...,Multiple,Airport Operations,2022-08-03,False,False,38.701,-121.595,1973-01-01,2023-09-03,72483993225
9,1278541,2018-12-20 20:00:00,12,2018,2023-09-20 20:00:00,Night,KJAX,JACKSONVILLE INTL,30.49406,-81.68786,...,Multiple,Pilot,2022-08-03,False,False,30.495,-81.694,1948-01-01,2023-09-03,72206013889


In [65]:
data_faa_ncei['INCIDENT_DATE'] = pd.to_datetime(data_faa_ncei['INCIDENT_DATE'], errors='coerce')

In [66]:
data_faa_ncei.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55527 entries, 0 to 55526
Columns: 107 entries, INDX_NR to USAF_WBAN
dtypes: bool(39), datetime64[ns](1), float64(23), int64(3), object(41)
memory usage: 30.9+ MB


In [67]:
data_faa_ncei

,INDX_NR,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME,TIME_OF_DAY,AIRPORT_ID,AIRPORT,LATITUDE,LONGITUDE,...,SOURCE,PERSON,LUPDATE,IMAGE,TRANSFER,NEAREST_STATION_LAT,NEAREST_STATION_LON,NEAREST_STATION_BEGIN,NEAREST_STATION_END,USAF_WBAN
0,818272,2018-12-20 10:10:00,12,2018,2023-09-20 10:10:00,Day,KSEA,SEATTLE-TACOMA INTL,47.44898,-122.30931,...,FAA Form 5200-7-E,Pilot,2019-03-18,False,False,47.445,-122.314,1948-01-01,2023-09-03,72793024233
1,818275,2018-12-20 12:07:00,12,2018,2023-09-20 12:07:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,FAA Form 5200-7-E,Pilot,2019-03-18,False,False,25.788,-80.317,1973-01-01,2023-09-03,72202012839
2,818288,2018-12-20 20:54:00,12,2018,2023-09-20 20:54:00,Night,KLAX,LOS ANGELES INTL,33.94254,-118.40807,...,FAA Form 5200-7-E,Airport Operations,2019-03-18,False,False,33.938,-118.387,1944-01-01,2023-09-03,72295023174
3,818618,2018-12-20 15:53:00,12,2018,2023-09-20 15:53:00,Day,KDAL,DALLAS LOVE FIELD ARPT,32.84711,-96.85177,...,MOR,Tower,2019-07-27,False,False,32.838,-96.836,1946-07-26,2023-09-03,72258013960
4,819264,2018-12-20 05:28:00,12,2018,2023-09-20 05:28:00,Night,KORD,CHICAGO O'HARE INTL ARPT,41.97960,-87.90446,...,MOR,Tower,2019-07-27,False,False,41.960,-87.932,1946-10-01,2023-09-03,72530094846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55522,1170484,2022-01-20 15:25:00,1,2022,2023-09-20 15:25:00,Day,PHNL,DANIEL K. INOUYE INTERNATIONAL AIRPORT,21.31869,-157.92241,...,FAA Form 5200-7-E,Pilot,2022-01-07,False,False,21.324,-157.939,1939-06-01,2023-09-03,91182022521
55523,1170487,2022-01-20 12:10:00,1,2022,2023-09-20 12:10:00,Day,KMIA,MIAMI INTL,25.79325,-80.29056,...,FAA Form 5200-7-E,Pilot,2022-01-07,True,False,25.788,-80.317,1973-01-01,2023-09-03,72202012839
55524,1170453,2022-01-20 09:25:00,1,2022,2023-09-20 09:25:00,NaN,KSDF,MUHAMMAD ALI INTERNATIONAL,38.17439,-85.73600,...,FAA Form 5200-7-E,Carcass Found,2022-01-07,True,False,38.177,-85.731,1973-01-01,2023-09-03,72423093821
55525,1170467,2022-01-20 21:15:00,1,2022,2023-09-20 21:15:00,NaN,KMCN,MIDDLE GEORGIA REGIONAL,32.69285,-83.64921,...,FAA Form 5200-7-E,Carcass Found,2022-01-07,False,False,32.689,-83.653,1941-10-20,2023-09-03,72217003813


### Выгрузка данных о метеостанциях

In [68]:
# Создаем пустой список для хранения данных
all_data = []

# Задаем список годов, которые мы хотим обработать
years = [2018, 2019, 2020, 2021, 2022]

# Проходим по каждому году
for year in years:
    # Фильтруем данные для текущего года
    data_faa_ncei_year = data_faa_ncei[data_faa_ncei['INCIDENT_DATE'].dt.year == year]

    # Получаем список уникальных станций для этого года
    stations = data_faa_ncei_year['USAF_WBAN'].unique()

    # Определяем базовый URL для запросов
    base_url = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-hourly&stations={station_name}&dataTypes=WND,CIG,VIS,TMP,DEW,SLP&startDate={start_date}&endDate={end_date}&includeAttributes=true&format=csv"

    # Создаем список URL-ссылок для запросов
    urls = []
    for station_name in stations:
        # Фильтруем данные для текущей станции
        station_data = data_faa_ncei_year[data_faa_ncei_year['USAF_WBAN'] == station_name]
        
        # Извлекаем минимальную и максимальную дату для этой станции
        min_date = station_data['INCIDENT_DATE'].min().strftime('%Y-%m-%dT00:00:00')
        max_date = station_data['INCIDENT_DATE'].max().strftime('%Y-%m-%dT23:59:59')
        
        # Заменяем параметры в базовом URL
        url = base_url.format(station_name=station_name, start_date=min_date, end_date=max_date)
        urls.append(url)

    # Отправляем запросы и сохраняем данные с отображением прогресса
    for url in tqdm(urls, desc=f"Загрузка данных за {year} год"):
        retry_count = 3  # Количество попыток

        while retry_count > 0:
            response = requests.get(url)
            if response.status_code == 200:
                # Разделяем данные на строки и добавляем их в наш список
                lines = response.text.split('\n')
                all_data.extend(lines)
                break
            else:
                print(f"Не удалось получить данные по URL {url}. Код состояния: {response.status_code}")
                retry_count -= 1
                if retry_count > 0:
                    print(f"Повторная попытка через 15 секунд...")
                    time.sleep(15)  # Задержка перед повторной попыткой
                else:
                    print("Исчерпаны все попытки запроса.")
                    break  # Выход из цикла в случае неудачных попыток

    # Задаем папку для сохранения файла
    save_folder = r'C:\Users\Lenovo\Desktop\DE\Проект про птиц\Сырые данные\Stations'

    # Создаем имя файла для текущего года
    file_name = f'all_stations_data_{year}.csv'
    file_path = os.path.join(save_folder, file_name)

    # Сохраняем данные в файл для текущего года
    with open(file_path, 'w', newline='') as csvfile:
        csvfile.write('\n'.join(all_data))

    print(f"Данные за {year} год сохранены в файл: {file_path}")

    # Очищаем all_data для следующего года
    all_data.clear()

print("Все данные сохранены в отдельные файлы по годам.")

Загрузка данных за 2018 год:   8%|████▎                                               | 73/891 [02:01<23:55,  1.75s/it]

Не удалось получить данные по URL https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-hourly&stations=72210812894&dataTypes=WND,CIG,VIS,TMP,DEW,SLP&startDate=2018-02-20T00:00:00&endDate=2018-12-20T23:59:59&includeAttributes=true&format=csv. Код состояния: 503
Повторная попытка через 15 секунд...


Загрузка данных за 2018 год:  22%|██████████▉                                        | 192/891 [06:26<21:47,  1.87s/it]

Не удалось получить данные по URL https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-hourly&stations=72235413927&dataTypes=WND,CIG,VIS,TMP,DEW,SLP&startDate=2018-06-20T00:00:00&endDate=2018-11-20T23:59:59&includeAttributes=true&format=csv. Код состояния: 503
Повторная попытка через 15 секунд...


Загрузка данных за 2018 год:  29%|██████████████▉                                    | 260/891 [09:31<18:53,  1.80s/it]

...

Не удалось получить данные по URL https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-hourly&stations=72394023273&dataTypes=WND,CIG,VIS,TMP,DEW,SLP&startDate=2022-10-20T00:00:00&endDate=2022-10-20T23:59:59&includeAttributes=true&format=csv. Код состояния: 503
Повторная попытка через 15 секунд...


Загрузка данных за 2022 год:  69%|██████████████████████████████████▉                | 527/769 [19:29<04:48,  1.19s/it]

Не удалось получить данные по URL https://www.ncei.noaa.gov/access/services/data/v1?dataset=global-hourly&stations=72586594161&dataTypes=WND,CIG,VIS,TMP,DEW,SLP&startDate=2022-06-20T00:00:00&endDate=2022-09-20T23:59:59&includeAttributes=true&format=csv. Код состояния: 503
Повторная попытка через 15 секунд...


Загрузка данных за 2022 год: 100%|███████████████████████████████████████████████████| 769/769 [23:44<00:00,  1.85s/it]


Данные за 2022 год сохранены в файл: C:\Users\Lenovo\Desktop\DE\Проект про птиц\Сырые данные\Stations\all_stations_data_2022.csv
Все данные сохранены в отдельные файлы по годам.


In [69]:
# Годы для обработки
years = [2018, 2019, 2020, 2021, 2022]

# Создаем пустые датасеты для каждого года
data_meteostation_2018 = pd.DataFrame()
data_meteostation_2019 = pd.DataFrame()
data_meteostation_2020 = pd.DataFrame()
data_meteostation_2021 = pd.DataFrame()
data_meteostation_2022 = pd.DataFrame()

# Цикл для загрузки и обработки файлов
for year in years:
    # Формируем имя файла
    file_name = f'all_stations_data_{year}.csv'
    file_path = os.path.join(save_folder, file_name)

    # Читаем файл CSV с отображением прогресса
    with tqdm(total=1, desc=f"Загрузка данных за {year} год") as pbar:
        data_year = pd.read_csv(file_path)
        pbar.update(1)  # Обновляем прогресс после загрузки файла

    # Удаляем указанные столбцы
    columns_to_drop = ["SOURCE", "REPORT_TYPE", "CALL_SIGN", "QUALITY_CONTROL"]
    data_year = data_year.drop(columns=columns_to_drop)

    # Преобразуем столбец "date" в нужный формат
    data_year['DATE'] = pd.to_datetime(data_year['DATE'], errors='coerce')  # Используем errors='coerce', чтобы обрабатывать ошибки

    # Удаляем строки с некорректными датами (которые не удалось преобразовать)
    data_year = data_year.dropna(subset=['DATE'])

    # Зависимо от года, загружаем данные в соответствующий датасет
    if year == 2018:
        data_meteostation_2018 = data_year
    elif year == 2019:
        data_meteostation_2019 = data_year
    elif year == 2020:
        data_meteostation_2020 = data_year
    elif year == 2021:
        data_meteostation_2021 = data_year
    elif year == 2022:
        data_meteostation_2022 = data_year

# Теперь у нас есть 5 разных датасетов, каждый из которых содержит данные за определенный год, с отображением прогресса загрузки и столбцом "date" в нужном формате.


Загрузка данных за 2021 год:   0%|                                                               | 0/1 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14544\2143906264.py:19: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  data_year = pd.read_csv(file_path)
Загрузка данных за 2022 год:   0%|                                                               | 0/1 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14544\2143906264.py:19: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  data_year = pd.read_csv(file_path)
Загрузка данных за 2022 год: 100%|███████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.75s/it]


In [77]:
data_meteostation_2018[data_meteostation_2018['USAF_WBAN'] == '72524014820']

,USAF_WBAN,DATE,CIG,DEW,SLP,TMP,VIS,WND
2970023,72524014820,2018-01-20 00:00:00,"22000,1,9,N","-0072,1","10146,1","+0022,1","016000,1,9,9","190,1,N,0088,1"
2970024,72524014820,2018-01-20 00:51:00,"22000,5,9,N","-0072,5","10139,5","+0022,5","016093,5,N,5","200,5,N,0077,5"
2970025,72524014820,2018-01-20 01:51:00,"07620,5,M,N","-0067,5","10138,5","+0028,5","016093,5,N,5","200,5,N,0082,5"
2970026,72524014820,2018-01-20 02:51:00,"22000,5,9,N","-0067,5","10132,5","+0028,5","016093,5,N,5","190,5,N,0088,5"
2970027,72524014820,2018-01-20 03:00:00,"22000,1,9,N","-0067,1","10132,1","+0028,1","016000,1,9,9","190,1,N,0088,1"
...,...,...,...,...,...,...,...,...
535192,72524014820,2018-12-20 20:51:00,"02896,5,M,N","+0017,5","10074,5","+0094,5","016093,5,N,5","080,5,N,0021,5"
535193,72524014820,2018-12-20 21:00:00,"99999,9,9,N","+0017,1","10074,1","+0094,1","016000,1,9,9","080,1,N,0021,1"
535194,72524014820,2018-12-20 21:51:00,"01829,5,M,N","+0017,5","10072,5","+0089,5","016093,5,N,5","050,5,N,0026,5"
535195,72524014820,2018-12-20 22:51:00,"02134,5,M,N","+0033,5","10057,5","+0072,5","016093,5,N,5","040,5,N,0036,5"


#### Объединение данных метеостанций

In [82]:
# Преобразование дат в datetime, если они еще не в этом формате
data_faa_ncei['INCIDENT_DATE'] = pd.to_datetime(data_faa_ncei['INCIDENT_DATE'])

# Сортировка данных по столбцу 'INCIDENT_DATE' в data_faa_ncei
data_faa_ncei.sort_values(by='INCIDENT_DATE', inplace=True)

# Создаем пустой DataFrame, который будет хранить итоговый результат
data_faa_ncei_stations = pd.DataFrame()

# Список имен датасетов data_meteostation_2018 до data_meteostation_2022
meteostation_datasets = ['data_meteostation_2018', 'data_meteostation_2019', 'data_meteostation_2020', 'data_meteostation_2021', 'data_meteostation_2022']

for dataset_name in meteostation_datasets:
    # Преобразование дат в datetime, если они еще не в этом формате
    globals()[dataset_name]['DATE'] = pd.to_datetime(globals()[dataset_name]['DATE'])
    
    # Сортировка данных по столбцу 'DATE' в текущем датасете
    globals()[dataset_name].sort_values(by='DATE', inplace=True)
    
    # Переименование столбца 'STATION' в 'USAF_WBAN' в текущем датасете
    globals()[dataset_name].rename(columns={'STATION': 'USAF_WBAN'}, inplace=True)
    
    # Объединение данных по ближайшей дате с текущим датасетом
    data_faa_ncei_stations = pd.merge_asof(data_faa_ncei, globals()[dataset_name], left_on='INCIDENT_DATE', right_on='DATE', by=['USAF_WBAN'], direction='nearest')
    
    # Удаление лишних столбцов, включая 'DATE', который больше не нужен
    data_faa_ncei_stations.drop('DATE', axis=1, inplace=True)

# data_faa_ncei_stations теперь содержит объединенные данные из всех пяти датасетов и назван соответствующим образом


In [84]:
data_faa_ncei_stations.to_csv('C:/Users/Lenovo/Desktop/DE/Проект про птиц/Выгрузка таблиц/FAA + NCEI/data_faa_ncei_stations.csv', index=False, sep=',', quoting=csv.QUOTE_ALL)

In [73]:
data_faa_ncei_stations

,INDX_NR,INCIDENT_DATE,INCIDENT_MONTH,INCIDENT_YEAR,TIME,TIME_OF_DAY,AIRPORT_ID,AIRPORT,LATITUDE,LONGITUDE,...,NEAREST_STATION_LON,NEAREST_STATION_BEGIN,NEAREST_STATION_END,USAF_WBAN,CIG,DEW,SLP,TMP,VIS,WND
0,820055,2018-01-20 00:13:00,1,2018,2023-09-20 00:13:00,Night,KCLE,CLEVELAND-HOPKINS INTL ARPT,41.410890,-81.849400,...,-81.852,1973-01-01,2023-09-03,72524014820,NaN,NaN,NaN,NaN,NaN,NaN
1,818968,2018-01-20 01:21:00,1,2018,2023-09-20 01:21:00,Night,KIND,INDIANAPOLIS INTL ARPT,39.717330,-86.294380,...,-86.282,1942-10-06,2023-09-03,72438093819,NaN,NaN,NaN,NaN,NaN,NaN
2,802088,2018-01-20 01:30:00,1,2018,2023-09-20 01:30:00,Night,SBGL,GALEAO INTL (RIO),-22.808903,-43.243647,...,-43.244,1931-01-01,2023-09-02,83746099999,NaN,NaN,NaN,NaN,NaN,NaN
3,1065093,2018-01-20 01:30:00,1,2018,2023-09-20 01:30:00,Day,KWRI,MC GUIRE AIR FORCE BASE,40.015582,-74.591708,...,-74.600,1942-08-01,2023-09-03,72409614706,NaN,NaN,NaN,NaN,NaN,NaN
4,818928,2018-01-20 01:59:00,1,2018,2023-09-20 01:59:00,Night,KIAH,GEORGE BUSH INTERCONTINENTAL/ HOUSTON ARPT,29.980470,-95.339720,...,-95.361,1973-01-01,2023-09-03,72243012960,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55522,1369759,2022-12-20 23:10:00,12,2022,2023-09-20 23:10:00,Night,KFAY,FAYETTEVILLE REGNL ARPT/GRANNIS FIELD,34.991470,-78.880000,...,-78.880,2006-01-01,2023-09-03,72303593740,"02286,5,M,N","-0028,5","10302,5","+0039,5","016093,5,N,5","030,5,N,0031,5"
55523,1370406,2022-12-20 23:15:00,12,2022,2023-09-20 23:15:00,Night,KMEM,MEMPHIS INTL,35.042420,-89.976670,...,-89.986,1946-10-01,2023-09-03,72334013893,"22000,5,9,N","-0028,5","10281,5","+0089,5","016093,5,N,5","010,5,N,0041,5"
55524,1370386,2022-12-20 23:20:00,12,2022,2023-09-20 23:20:00,Night,KBWI,BALTIMORE/WASH INTL THURGOOD MARSHAL ARPT,39.175400,-76.668200,...,-76.684,1945-01-01,2023-09-03,72406093721,"07620,5,M,N","-0061,5","10308,5","+0022,5","016093,5,N,5","310,5,N,0015,5"
55525,1370418,2022-12-20 23:30:00,12,2022,2023-09-20 23:30:00,Night,KLBB,LUBBOCK PRESTON SMITH INTL ARPT,33.663640,-101.822780,...,-101.823,1945-08-10,2023-09-03,72267023042,"07620,5,M,N","-0006,5","10279,5","+0017,5","016093,5,N,5","140,5,N,0057,5"


## Выгрузка BTS

Посчитаем топ 10 аэропортов по крушениям

In [ ]:
data_faa_full.head(10)

In [ ]:
data_faa_full['AIRPORT'].value_counts()

In [ ]:
data_faa_full[data_faa_full['AIRPORT'] == 'UNKNOWN']

Удалим значения UNKNOWN из столбца "AIRPORT" так как они помешают при подсчете иницидентов

In [ ]:
# Преобразование дат в datetime, если они еще не в этом формате
data_faa_ncei_2018['INCIDENT_DATE'] = pd.to_datetime(data_faa_ncei_2018['INCIDENT_DATE'])
data_meteostation_2018['DATE'] = pd.to_datetime(data_meteostation_2018['DATE'])

# Переименование столбца 'STATION' в 'USAF_WBAN' в data_meteostation_2018
data_meteostation_2018.rename(columns={'STATION': 'USAF_WBAN'}, inplace=True)

# Создание пустого датафрейма для хранения ближайших метеоданных
nearest_weather_data_2018 = pd.DataFrame(columns=['USAF_WBAN', 'INCIDENT_DATE', 'CIG', 'DEW', 'SLP', 'TMP', 'VIS', 'WND'])

# Итерация по инцидентам и поиску ближайших дат и времени в метеоданных с прогрессом
for index, incident_row in tqdm(data_faa_ncei_2018.iterrows(), total=len(data_faa_ncei_2018), desc="Обработка инцидентов"):
    incident_date = incident_row['INCIDENT_DATE']
    station_id = incident_row['USAF_WBAN']
    
    # Фильтрация метеоданных по станции
    station_data = data_meteostation_2018[data_meteostation_2018['USAF_WBAN'] == station_id]
    
    if not station_data.empty:
        # Нахождение ближайшей даты и времени в метеоданных
        nearest_date_idx = np.argmin(np.abs(station_data['DATE'] - incident_date))
        nearest_weather_row = station_data.iloc[nearest_date_idx]
        
        # Добавление найденных данных в датафрейм с помощью concat
        nearest_weather_data_2018 = pd.concat([nearest_weather_data_2018, nearest_weather_row.to_frame().T], ignore_index=True)
    else:
        # Если для данной станции нет данных, добавляем NaN или другие значения по умолчанию
        temp_df = pd.DataFrame({'USAF_WBAN': [station_id], 'INCIDENT_DATE': [incident_date]})
        nearest_weather_data_2018 = pd.concat([nearest_weather_data_2018, temp_df], ignore_index=True)

# Результат будет храниться в датафрейме nearest_weather_data_2018

In [ ]:
data_bts = data_faa_full[data_faa_full['AIRPORT'] != 'UNKNOWN']

In [ ]:
data_top10 = data_bts.groupby(['AIRPORT', 'AIRPORT_ID'])['INCIDENT_YEAR'].count().reset_index()

# Переименуем столбец 'INCIDENT_YEAR' в 'INCIDENT_COUNT'
data_top10 = data_top10.rename(columns={'INCIDENT_YEAR': 'INCIDENT_COUNT'})

# Теперь столбец 'INCIDENT_YEAR' переименован в 'INCIDENT_COUNT'

In [ ]:
top_10_airports = data_top10.sort_values('INCIDENT_COUNT', ascending=False).head(10)
top_10_airports

Определили ТОП-10 аэропортов для дальнейшей выгрузки данных

Откроем все 10 датасэта для аэропортов

### Выгрузка данных

In [ ]:
# Путь к каталогу с файлами
directory_path_bts = r'C:/Users/Lenovo/Desktop/DE/Проект про птиц/Сырые данные/BTS'

# Получим список файлов в указанном каталоге
file_list = os.listdir(directory_path_bts)

# Создаём пустой словарь, где ключами будут имена файлов, а значениями - соответствующие DataFrame
data_dict = {}

# Пройдём по списку файлов и загрузим данные в словарь
for file_name in file_list:
    file_path = os.path.join(directory_path_bts, file_name)
    # Используем имя файла (без расширения) в качестве ключа в словаре
    file_name_without_extension = os.path.splitext(file_name)[0]
    
    # Пропустим первую строку и последние две строки при чтении файла
    data = pd.read_csv(file_path, skiprows=1, skipfooter=2, engine='python')
    
    # Обрабатываем значения в указанных столбцах
    columns_to_process = ['DOMESTIC', 'INTERNATIONAL', 'TOTAL']
    
    for column in columns_to_process:
        # Создадим пустой список для хранения обработанных значений
        processed_values = []

        # Перебираем каждое значение в столбце
        for value in data[column]:
            if ',' in str(value):
                # Если в значении есть запятая, определяем количество цифр после запятой
                num_decimal_places = len(str(value).split(',')[-1])

                # В зависимости от количества цифр после запятой, выполняем преобразование
                if num_decimal_places == 1:
                    processed_value = float(value.replace(',', '')) * 100
                elif num_decimal_places == 2:
                    processed_value = float(value.replace(',', '')) * 10
                else:
                    processed_value = float(value.replace(',', ''))
            else:
                # Если запятой нет, оставляем значение как есть
                processed_value = float(value)

            # Добавляем обработанное значение в список
            processed_values.append(processed_value)

        # Заменяем старый столбец на новый с обработанными значениями
        data[column] = processed_values
    
    # Добавляем обработанные данные в словарь
    data_dict[file_name_without_extension] = data

# Теперь у нас есть словарь с данными из файлов, и значения в указанных столбцах DOMESTIC, INTERNATIONAL и TOTAL
# обработаны в соответствии с вашими условиями, а также названия столбцов приведены к нижнему регистру.

In [ ]:
for dataset_name, dataset in data_dict.items():
    variable_name = f'data_{dataset_name}'
    globals()[variable_name] = dataset

In [ ]:
for dataset_name in data_dict.keys():
    dataset = globals()[f'data_{dataset_name}']
    dataset['AIRPORT_ID'] = dataset_name
    dataset = dataset[(dataset['Month'] != 'TOTAL') & (dataset['Year'] >= 2018) & (dataset['Year'] <= 2022)]
    globals()[f'data_{dataset_name}'] = dataset

In [ ]:
data_KATL

In [ ]:
all_datasets = []

# Пройдем по каждому датасету и добавьте его в список
for dataset_name in data_dict.keys():
    dataset = globals()[f'data_{dataset_name}']
    all_datasets.append(dataset)

# Объединим все датасеты в один с помощью метода concat
bts_dataset = pd.concat(all_datasets, ignore_index=True)

In [ ]:
bts_dataset

In [ ]:
# Получим текущую дату и время
current_datetime = datetime.datetime.now()

# Форматируем дату и время в строку в нужном формате (например, ГГГГММДД_ЧЧММСС)
formatted_datetime = current_datetime.strftime('%Y%m%d_%H%M%S')

# Сгенерируем имя файла, добавив его к "BTS_" и текущей дате и времени
output_file_name = f'BTS_{formatted_datetime}.csv'

# Укажем путь
directory_path = r'C:/Users/Lenovo/Desktop/DE/Проект про птиц/Выгрузка таблиц/BTS'

# Путь к файлу для сохранения
output_file_path = os.path.join(directory_path, output_file_name)

# Переведем все названия столбцов в нижний регистр для удобство использования
bts_dataset.columns = bts_dataset.columns.str.lower()

# Сохраним объединенный датасет в CSV-файл с сгенерированным именем
bts_dataset.to_csv(output_file_path, index=False)